## Introduction

This notebook is a component of the GEO712 course's final project at the School of Earth, Environment & Society of McMaster University. The main objective of the final project is to analyze and compare job accessibility levels across different spatial patterns, focusing on the Greater Golden Horseshoe area (GGH) in Ontario, Canada.

Within this notebook, we present a methodology for extracting features from satellite imagery and road network data and transferring them onto a hexagonal grid. The input files for this notebook include:
-   Hexagonal grid _(hexagon_500m2.shp)_: A cell grid comprised of hexagons with an area of 500 m². Remote sensing imagery and road network data attributes will be transferred to this grid.
-   Road network _(roads_osm_cl.shp)_: A shapefile representing the road network, which will be used to create variables and transfer them to the hexagonal grid.
-   Road intersections _(intersections_points.shp)_: A point shapefile indicating road intersections. This file will be used to create variables and transfer them to the hexagonal grid.
-   Distance matrix _(grid_distance_matrix.csv)_: A CSV-formatted table containing a distance matrix for the cell grid. 
-   Spatial Access Measures _(SAM, INCLUDE HERE)_: A national dataset in CSV format that measures accessibility to various opportunities, considering different modes of transport.
-   Dissemination block _(INCLUDE HERE)_: The spatial unit used by the SAM project to compute accessibility measures.

The output from this notebook includes shapefiles containing the features in the cell grid.

## Quarto

Load python libraries used in the notebook:

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from geopandas.tools import sjoin
import rasterio
from rasterstats import zonal_stats
import shapely
from shapely.geometry import Polygon, LineString

Reading the cell grid and creating a GeoDataFrame. We generate this hexagonal grid using QGIS software, indicating a value of 0.8774 meters to the principal diagonal, resulting in a hexagon with 500 m².

In [ ]:
grid = gpd.read_file("vectors\\hexagon_500m2.shp", geometry = 'geometry')

Creating a spatial index for the hexagonal grid to expedite spatial relations between geometries:

In [ ]:
grid.sindex

### Remote sensing attributes

Setting the satellite images' path: 

In [ ]:
raster_list = {'BP':'GHS_BUILT_S_E2018_epsg26917.tif', 'HE': 'GHS_BUILT_H_E2020_epsg26917.tif', 'VO': 'GHS_BUILT_V_E2020_epsg26917.tif', 'CL': 'GHS_BUILT_C_FUN_E2018_epsg26917.tif',
            'TR':'PROBAV_LC100_2018_Tree_epsg26917.tif',
            'CR':'PROBAV_LC100_2018_Crops_epsg26917.tif',
            'GR':'PROBAV_LC100_2018_Grass_epsg26917.tif',
            'WA':'PROBAV_LC100_2018_Water_epsg26917.tif',
            'SH':'PROBAV_LC100_2018_Shrub_epsg26917.tif'}

Extracting attributes from the satellite images to the hexagonal grid:

In [ ]:
new_grid_rs = grid

for key in raster_list.keys():
    path_image = 'rasters' + '\\' + raster_list[key]
   
    if key in ['BP', 'HE', 'VO', 'TR', 'GR','CR', 'WA', 'SH']:
        zs = zonal_stats(grid, path_image, stats=['min', 'max', 'mean', 'std', 'median', 'majority', 'sum'])
        df = pd.DataFrame(zs).fillna(0)
        df.columns = key + '_' + df.columns
        new_grid_rs = new_grid_rs.merge(df, left_index=True, right_index=True)    
       
    elif key in ['CL']:
        
        count = zonal_stats(grid, path_image, stats=['count'])

        if  key == 'CL':
            cmap = {0: 'NRC', 1: 'RES', 2:'COM'}
            zs = zonal_stats(grid, path_image, categorical = True, category_map=cmap)
            df = pd.DataFrame(zs).fillna(0)
            total = df['NRC'] + df['RES'] + df['COM']
            df['NRC'] = df['NRC']/total
            df['RES'] = df['RES']/total
            df['COM'] = df['COM']/total
            new_grid_rs = new_grid_rs.merge(df, left_index=True, right_index=True)

Saving the data so we can continue from this part next time.

In [ ]:
new_grid_rs.to_file("vectors\\rs_attributes.shp")
# new_grid_rs = gpd.read_file("vectors\\rs_attributes.shp")

### Road network attributes

Reading the road network shapefile and creating a GeoDataFrame.

In [ ]:
roads = gpd.read_file("vectors\\roads_osm_cl.shp")

Creating a spatial index for the road network to expedite spatial relations between geometries:

In [ ]:
roads.sindex

Extracting attributes from the road network to the hexagonal grid:

In [ ]:
explode = gpd.overlay(grid, roads, how='intersection', keep_geom_type=False).explode()
explode.index.names = ['level_1', 'level_2']
explode['SI'] = 0.0

for (level_1, level_2), group in explode.groupby(level=['level_1', 'level_2']):
        try:
            start_pt = LineString(explode['geometry'][level_1][level_2]).coords[0]
            end_pt = LineString(explode['geometry'][level_1][level_2]).coords[-1]
            straight_line = LineString((start_pt, end_pt))
            line = LineString(explode['geometry'][level_1][level_2])

            if straight_line.length == 0:
                sinuosity = 1
            else:
                sinuosity = line.length / (straight_line.length + 0.01)

            explode['SI'][level_1][level_2] = sinuosity 
        
        except:
                explode['SI'][level_1][level_2] = explode['SI'][level_1][level_2 - 1]
        
explode['LE'] = explode['geometry'].length

# Lista de atributos a serem transferidos
attributes_to_transfer = ['SI', 'LE']

# DataFrame para armazenar os resultados ponderados
weighted_values = pd.DataFrame()

for attribute in attributes_to_transfer:
    # Criar coluna ponderada para o atributo atual
    explode[f'{attribute}_weighted'] = explode[attribute] * explode['LE']

    # Calcular a soma ponderada por 'id'
    sum_weighted = explode.groupby('id')[f'{attribute}_weighted'].sum()

    # Calcular a soma total dos comprimentos por 'id'
    sum_length = explode.groupby('id')['LE'].sum()

    # Calcular a média ponderada e atribuir ao DataFrame final
    weighted_values[attribute + '_mean'] = sum_weighted / sum_length
    weighted_values[attribute + '_sum'] = explode.groupby('id')[attribute].sum()
    weighted_values[attribute + '_min'] = explode.groupby('id')[attribute].min()
    weighted_values[attribute + '_max'] = explode.groupby('id')[attribute].max()
    weighted_values[attribute + '_std'] = explode.groupby('id')[attribute].std()
    
weighted_values['NRo'] = explode.groupby('id')['LE'].count()

# Adicionar as colunas ponderadas ao DataFrame original (s_grid)
new_grid_roads = pd.merge(grid, weighted_values, left_on='id', right_index=True, how='left').fillna(0)

Saving the data so we can continue from this part next time.

In [ ]:
new_grid_roads.to_file("vectors/roads_attributes.shp")
#new_grid_roads = gpd.read_file("vectors/roads_attributes.shp")

Reading the roads intersections shapefile and creating a GeoDataFrame with spatial. After this, we'll do a spatial join between the intersection roads points and the hexagonal grid.

In [ ]:
intersections = gpd.read_file("vectors\\intersections_points.shp")
intersections.sindex
intersections['NIn'] = 0
it = pd.DataFrame(sjoin(grid, intersections, how="left", op='intersects').groupby('id')['NIn'].count())
del intersections

Bringing road attributes to the point intersections file to create new attributes.

In [ ]:
it['id'] = it.index
it.index.names = ['']
it = it.merge(new_grid_roads[['id','NRo']], left_on='id', right_on ='id', how='left').merge(new_grid_rs[['id','BP_sum','geometry']] ,left_on='id', right_on ='id', how='left')
it = gpd.GeoDataFrame(it)

Creating new variables for the intersection file.

In [ ]:
it['DInBP'] = it['NIn'] / (it['BP_sum'] + 0.1)
it['DInRo'] = it['NIn']  / it['NRo']

Saving the data so we can continue from this part next time.

In [ ]:
it.to_file("vectors/grade_with_intersections_points.shp")
# it = gpd.read_file("vectors/grade_with_intersections_points.shp")

Reorganizing the intersection GeoDataFrame columns. 

In [ ]:
it = it[['id','NIn','NRo', 'BP_sum', 'DInBP','DInRo', 'geometry']]

Creating a new GeoDataFrame with all attributes.

In [ ]:
grid_attributes = new_grid_rs.iloc[:,:-1].merge(new_grid_roads.iloc[:,:-1].merge(it[['id','NIn','DInBP','DInRo','geometry']], left_on = 'id', right_on = 'id', how="left"), left_on = 'id', right_on = 'id', how = 'left')

Filling missing values with 0.

In [ ]:
grid_attributes = grid_attributes.fillna(0)

### Neighboors attributes

Reading the grid distance matrix. This matrix was generated using QGIS software, indicating the six nearest target cells for each hexagon. 

In [ ]:
matrix = pd.read_csv("sheets\grid_distance_matrix.csv")

Merging the distance matrix with the grid GeoDataFrame with attributes.

In [ ]:
id_origem = 'InputID'
id_destin = 'TargetID'
data = matrix.merge(grid_attributes, left_on =id_destin, right_on='id')

Obtaining the neighbors' means for each grid attribute.

In [ ]:
NM = data.groupby(id_origem, sort=True).mean()
NM.insert(0,'InputID',NM.index)
NM['id'] = NM[id_origem]
NM.index.names = ['']
NM = NM.reset_index()
NM = NM[new_grid_rs.iloc[:,:-1].columns]
NM

Obtaining the normalized difference neighbors' means for each grid attribute.

In [ ]:
DNM = (pd.DataFrame(new_grid_rs.iloc[:,:-1]) - pd.DataFrame(NM))/(pd.DataFrame(new_grid_rs.iloc[:,:-1]) + pd.DataFrame(NM))
DNM = DNM.fillna(0)
DNM['id'] = NM['id']
DNM

Renaming the GeoDataFrame columns.

In [ ]:
DNM.columns = 'DNM_' + DNM.columns
NM.columns = 'NM_' + NM.columns

Merging GeoDataFrames: 

In [ ]:
DNM = DNM.merge(grid, right_on = 'id', left_on = 'DNM_id', how = 'left')
DNM = DNM.drop('id', axis = 1)
DNM

Creating a GeoDataFrame with all attributes:

In [ ]:
final_grid = grid_attributes.iloc[:,:-1].merge(NM, left_on = 'id', right_on = 'NM_id', how = 'left').merge(DNM, left_on = 'id', right_on = 'DNM_id', how = 'left')
final_grid = final_grid.drop(['NM_id','DNM_id'], axis = 1)
final_grid

Saving the final GeoDataFrame:

In [ ]:
gpd.GeoDataFrame(final_grid).to_file("vectors/grid_with_all_attributes.shp")
# final_grid = gpd.read_file("vectors/grid_with_all_attributes.shp")